<a href="https://colab.research.google.com/github/huskeypm/cellmigration/blob/main/master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# master
Workhorse for generating and executing jobs in support of migration paper.

Once the csv files are made from the postprocessing, use the colab notebook to plot:




In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
!git clone https://github.com/huskeypm/cellmigration
#!cd cellmigration; git pull

Cloning into 'cellmigration'...
remote: Enumerating objects: 693, done.
remote: Counting objects: 100% (183/183), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 693 (delta 109), reused 103 (delta 55), pack-reused 510
Receiving objects: 100% (693/693), 10.52 MiB | 12.40 MiB/s, done.
Resolving deltas: 100% (446/446), done.


In [ ]:
import sys
sys.path.append("/content/cellmigration/")


from google.colab import drive
gDrive='/content/drive/'
drive.mount(gDrive)
gDrive += "MyDrive/temp/231212"




Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import yaml
import sys

sys.path.append('/content/cellmigration/paper/')
import writeInputs as wi
#import batchProcess as bp
#outDir = "/home/pkh-lab-shared/migration/231110/"
outDir = "/home/pkekeneshuskey/data/231212/"
outDir = gDrive
# FOR LOCAL
cmd = "python3 brownian_v3.py -yamlFile " # if local

# FOR FAUST/W
cmd='$EXEC ' # if remote host

wi.params = wi.Params(
    cmd=cmd,
    path=outDir
)

print(wi.params.cmd)
print(wi.params.path)

varIter = 7 # keys
varRuns = 5 # replicates per run



ADJ PATH LATER
$EXEC 
/content/drive/MyDrive/temp/231212


The idea of the reference file is to have a complete system that toggled - for instance, we have a case that is tuned for ATP application, but we can set the ATP concentration to zero to emulate a ATP-free condition. Also want values that are intermediate, so when we vary them by 50%-200%, we get reasonable numbers

In [ ]:
referenceFile="cellmigration/paper/template_full.yaml"
with open(referenceFile, 'r') as file:
     daVars = yaml.safe_load(file)

## crowder free case
### No ATP
Can copy/paste into file to run


In [ ]:
nocrowders = daVars.copy()
nocrowders['nCrowders']=1
nocrowders['crowderRad']=0.1
nocrowders['outName']='nocrowder'
keys=['nCells','cellRad','cellAttr']

#nocrowders

In [ ]:
nocrowders_nocontainment = nocrowders.copy()
nocrowders_nocontainment['containmentPotential']=False
nocrowders_nocontainment['nUpdates']=10000
nocrowders_nocontainment['absorbingBoundary']=False
nocrowders_nocontainment['xPotential']=True
nocrowders_nocontainment['outName']= 'nocrowder_nocontainment'
key=['xScale']
varItersList = [0] # only one conc. makes sense ,1.,2.,3.,4.,5.,6.,7.]

daKey = key[0]
print("Generate ",daKey)
writeFile = nocrowders_nocontainment['outName']+"_"+daKey+".bash"
wi.WriteIterativeYaml(nocrowders_nocontainment,daKey,varIter=varItersList, varRuns=varRuns,
                      writeFile=writeFile)

Generate  xScale
Wrote nocrowder_nocontainment_xScale.bash


In [ ]:

####### cellAttr ----> particleAttr
for daKey in keys:
    print("Generate ",daKey)
    if "nCells" in daKey:
      varItersList = [30,40,50,60,70,80,90]
    else:
      varItersList = varIter
    writeFile = nocrowders['outName']+"_"+daKey+".bash"
    wi.WriteIterativeYaml(nocrowders,daKey,varIter=varItersList, varRuns=varRuns,
                         writeFile=writeFile)


Generate  nCells
Wrote nocrowder_nCells.bash
Generate  cellRad
cellRad [2.         2.5198421  3.1748021  4.         5.0396842  6.34960421
 8.        ]
Wrote nocrowder_cellRad.bash
Generate  cellAttr
cellAttr [0.05       0.06299605 0.07937005 0.1        0.1259921  0.15874011
 0.2       ]
Wrote nocrowder_cellAttr.bash


In [ ]:
# generally too slow to run in notebook
#bp.processYamls(nocrowders['outName'],yamlNamePrefix=nocrowders['outName'],prefixOptions=keys,
#               display=False)

### + ATP



In [ ]:
nocrowdersatp = daVars.copy()
nocrowdersatp['nCrowders']=1
nocrowdersatp['crowderRad']=0.1
nocrowdersatp['outName']='nocrowder_atp'
nocrowdersatp['xPotential']=True
nocrowdersatp['yPotential']=True  # default yScale should be 0
keys=['nCells','cellRad','cellAttr','xScale','yScale']
print(wi.params.path)

/content/drive/MyDrive/temp/231212


In [ ]:


for daKey in keys:
    print("Generate ",daKey)
    if "nCells" in daKey:
      varItersList = [30,40,50,60,70,80,90]
    elif "xScale" in daKey:
      varItersList = [0,1.,2.,3.,4.,5.,6.,7.]
    elif "yScale" in daKey:
      varItersList = [0,1.,2.,3.,4.,5.,6.,7.]
is it worth doing negative as well to show repulsion
    else:
      varItersList = varIter
    writeFile = nocrowdersatp['outName']+"_"+daKey+".bash"
    wi.WriteIterativeYaml(nocrowdersatp,daKey,varIter=varItersList, varRuns=varRuns,
                         writeFile=writeFile)



Generate  nCells
Wrote nocrowder_atp_nCells.bash
Generate  cellRad
cellRad [2.         2.5198421  3.1748021  4.         5.0396842  6.34960421
 8.        ]
Wrote nocrowder_atp_cellRad.bash
Generate  cellAttr
cellAttr [0.05       0.06299605 0.07937005 0.1        0.1259921  0.15874011
 0.2       ]
Wrote nocrowder_atp_cellAttr.bash
Generate  xScale
Wrote nocrowder_atp_xScale.bash
Generate  yScale
Wrote nocrowder_atp_yScale.bash


In [ ]:
#bp.processYamls(nocrowdersatp['outName'],
#                yamlNamePrefix=nocrowdersatp['outName'],prefixOptions=keys,
#                display=False
#               )

## crowder  case
### No ATP

In [ ]:
crowders = daVars.copy()
crowders['outName']='crowder'
keys=['nCrowders','crowderRad','crowderAttr']
print(wi.params.path)

/content/drive/MyDrive/temp/231212


In [ ]:
for daKey in keys:
    print("Generate ",daKey)
    writeFile = crowders['outName']+"_"+daKey+".bash"
    if daKey == 'nCrowders':
      varItersList = [1,4,9,16,18]
    elif daKey == 'crowderRad':
      varItersList = [0.1,1,3,5,7,9,10]
    elif daKey == 'crowderAttr':
      varItersList = [0.1,1,3,5,7,9,10] (do this? testing up to 6 right now  )
    else:
      varItersList= varIter
    wi.WriteIterativeYaml(crowders,daKey,varIter=varItersList, varRuns=varRuns,
                         writeFile=writeFile)

Generate  nCrowders
Wrote crowder_nCrowders.bash
Generate  crowderRad
Wrote crowder_crowderRad.bash
Generate  crowderAttr
crowderAttr [0.05       0.06299605 0.07937005 0.1        0.1259921  0.15874011
 0.2       ]
Wrote crowder_crowderAttr.bash


In [ ]:
#bp.processYamls(crowders['outName'],yamlNamePrefix=crowders['outName'],prefixOptions=keys)

### +ATP

In [ ]:
crowdersatp = daVars.copy()
crowdersatp['outName']='crowder_atp'
crowdersatp['xPotential']=True
keys=['nCrowders','crowderRad','crowderAttr']
print(wi.params.path)

/content/drive/MyDrive/temp/231212


In [ ]:
for daKey in keys:
    print("Generate ",daKey)
    writeFile = crowdersatp['outName']+"_"+daKey+".bash"
    if daKey == 'nCrowders':
      varItersList = [1,4,9,16,18]
    elif daKey == 'crowderRad':
      varItersList = [0.1,1,3,5,7,9,10]
    #elif "xScale" in daKey:
    #  varItersList = [0,1.,2.,3.,4.,5.,6.,7.]
    else:
      varItersList = 7
    wi.WriteIterativeYaml(crowdersatp,daKey,varIter=varItersList, varRuns=varRuns,
                         writeFile=writeFile)

Generate  nCrowders
Wrote crowder_atp_nCrowders.bash
Generate  crowderRad
Wrote crowder_atp_crowderRad.bash
Generate  crowderAttr
crowderAttr [0.05       0.06299605 0.07937005 0.1        0.1259921  0.15874011
 0.2       ]
Wrote crowder_atp_crowderAttr.bash


In [ ]:
# bp.processYamls(crowdersatp['outName'],yamlNamePrefix=crowdersatp['outName'],prefixOptions=keys)

## Move bash files

In [ ]:
mv *sh /content/drive/MyDrive/temp/231212/